<a href="https://colab.research.google.com/github/elsonyeh/computer-programming-and-application/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 依使用者輸入：sticker, image, video, loaction，分別傳送貼圖、圖片、影片及地點，否則回覆 Gemini 的結果。

In [2]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 12.4 MB/s eta 0:00:00


In [4]:
# for colab
from google.colab import userdata
from flask import Flask, request, abort
from pyngrok import ngrok

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage, StickerMessage, ImageMessage, VideoMessage, LocationMessage
)

import google.generativeai as genai

# 讀取 Secret
LINE_CHANNEL_ACCESS_TOKEN = userdata.get("LINE_CHANNEL_ACCESS_TOKEN")
LINE_CHANNEL_SECRET = userdata.get("LINE_CHANNEL_SECRET")
NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

# 初始化 Flask
app = Flask(__name__)

# 初始化 Linebot
configuration = Configuration(access_token=LINE_CHANNEL_ACCESS_TOKEN)
handler = WebhookHandler(LINE_CHANNEL_SECRET)

# 初始化 Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel(model_name="gemini-2.0-flash")

def ask_gemini(question):
    response = model.generate_content(question)
    return response.text

# Webhook 路由
@app.route("/callback", methods=["POST"])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return "OK"

# 處理收到的 Line 訊息
@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == "sticker":
            reply = StickerMessage(package_id="446", sticker_id="1989")
        elif action == "image":
            reply = ImageMessage(
                original_content_url="https://cdn.pixabay.com/photo/2019/02/21/18/36/city-4011922_1280.jpg",
                preview_image_url="https://cdn.pixabay.com/photo/2019/02/21/18/36/city-4011922_1280.jpg"
            )
        elif action == "video":
            reply = VideoMessage(
                original_content_url="https://cdn.pixabay.com/video/2021/08/07/60742-573496615_large.mp4",
                preview_image_url="https://cdn.pixabay.com/photo/2017/11/14/05/41/taipei-taiwan-2947814_1280.jpg"
            )
        elif action == "location":
            reply = LocationMessage(
                title="台北101",
                address="台北市信義路五段7號",
                latitude=25.03409570979266,
                longitude=121.564536536767
            )
        else:
            response = ask_gemini(action)
            reply = TextMessage(text=response)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[reply]
            )
        )

# 啟動 pyngrok + Flask
if __name__ == "__main__":
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(5000).public_url
    print("👉 請把這個網址填到 LINE Developer Webhook URL：")
    print(f"{public_url}/callback")

    app.run(host="0.0.0.0", port=5000)


👉 請把這個網址填到 LINE Developer Webhook URL：
https://715b-34-48-197-44.ngrok-free.app/callback
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
